# 1.2 Summarizing Variables

In the previous section, we emphasized the difference between quantitative and categorical variables. The distinction is not merely pedantic; `pandas` will actually behave differently depending on whether it thinks a variable is quantitative or categorical.

It is not easy for a human to make sense of _all_ the values of a variable. In this section, we focus on ways to reduce the values to just a handful of summary statistics. Our working example will again be the Titanic data set, which contains both quantitative and categorical variables.

In [1]:
import pandas as pd
pd.options.display.max_rows = 8

df = pd.read_csv("titanic.csv")
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.50,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.00,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3,0,"Zimmerman, Mr. Leo",male,29.00,0,0,315082,7.8750,NaN,S,NaN,NaN,NaN


To get a quick summary of a variable, we can use the `.describe()` function. Let's see what happens when we call `.describe()` on a quantitative variable, like `age`.

In [2]:
df.age.describe()

count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: age, dtype: float64

It returns the count (the number of observations with non-missing values), the mean, the standard deviation (`std`), and various percentiles (`min`, `25%`, `50%`, `75%`, `max`).

Now, what if we call `.describe()` on a categorical variable, like `embarked`? This is a variable that takes on the values `C`, `Q`, or `S`, depending on whether the passenger embarked at Cherbourg, Queenstown, or Southampton.

In [3]:
df.embarked.describe()

count     1307
unique       3
top          S
freq       914
Name: embarked, dtype: object

The description of this variable is very different. We still get the count (of non-missing values). But instead of the mean and standard deviation (how would you calculate the mean of `Q` and `S`, anyway?), we get the number of unique values (`unique`), the value that appeared most often (`top`), and how often it appeared (`freq`). These are more natural summaries for a categorical variable, which only take on a limited set of values, where the values are often not even numeric.

The `.describe()` function only provides a handful of the many summary statistics that are available in `pandas`. We extract additional summary statistics below.

## Summary Statistics for Quantitative Variables

What statistics should we use to summarize a quantitative variable? The most salient features of a quantitative variable are its **center** and **spread**.

### Measures of Center

Some statistics measure the **center** of a variable. Two commonly used measures of the center are:

- the **mean** (a.k.a. average): the sum of the values divided by the count
- the **median**: the middle value when you sort the values (i.e., a value such that 50% of the values lie below and 50% of the values lie above)

A measure of center gives us information about the "typical" value of a variable. For example, you might not know whether a typical fare on the Titanic was £1, £10, or £100. But if we calculate the mean:

In [4]:
# The huge fares are affecting the mean  
# mean is sensitive to other things as well
df.fare.mean()

33.29547928134572

we see that a typical fare is around £30.

Let's see what the median says about the "typical" fare:

In [5]:
df.fare.median()

14.4542

The median is quite different from the mean! It says that about 50% of the passengers paid less than £15 and about 50% paid more, so another reasonable value for the "typical" fare is £15.

The mean was twice the median! What explains this discrepancy? The reason is that the mean is very sensitive to extreme values. To see this, let's look at the highest fare that any passenger paid.

In [6]:
df.fare.max()

512.3292

The highest fare paid was over £500! Even if most passengers paid less than £15, extreme values like this one will drag the mean upward. On the other hand, since the median is always the middle value, it is not affected by the extreme values, as long as the ordering of the values is not changed.

To drive this point home, let's see what would happen to the mean and median if that maximum fare were actually £10,000.

In [7]:
fare_10k = df.fare.replace(df.fare.max(), 10000)
fare_10k.mean(), fare_10k.median()

(62.30976307339481, 14.4542)

Notice how the mean is now over £60, but the median is unchanged.

Just to satisfy our curiosity, let's learn more about this passenger who paid the maximum fare. To do this, we have to find the row that achieved this maximum value. Fortunately, there is a convenient `pandas` function, `.idxmax()`, that returns the _row index_ of the maximum fare. (A mathematician might call this the ["arg max"](https://en.wikipedia.org/wiki/Arg_max).)

In [8]:
df.fare.idxmax()

49

Now we can select the row corresponding to this index using `.loc`, as we learned in the previous section.

In [49]:
df.loc[df.fare.idxmax()]

pclass                                                    1
survived                                                  1
name                     Cardeza, Mr. Thomas Drake Martinez
sex                                                    male
age                                                      36
sibsp                                                     0
parch                                                     1
ticket                                             PC 17755
fare                                                512.329
cabin                                           B51 B53 B55
embarked                                                  C
boat                                                      3
body                                                    NaN
home.dest    Austria-Hungary / Germantown, Philadelphia, PA
Name: 49, dtype: object

The median is a number below which 50% of the values fall. What if we want to know some other percentile? We can use the `.quantile()` function, which takes a percentile rank (between 0 and 1) as input and returns the corresponding percentile.

For example, the 75th percentile is:

In [10]:
df.fare.quantile(.75)

31.275

which is pretty close to the mean. So only about 25% of the passengers paid more than the mean! The mean is not a great measure of center when there are extreme values, as in this data set.

To summarize, we have encountered several `pandas` functions that can be used to summarize a quantitative variable:

- `.mean()` calculates the mean or average.
- `.median()` calculates the median.
- `.quantile(q)` returns a value such that a fraction `q` of the values fall below that value (in other words, the (100q)th percentile).
- `.max()` calculates the maximum value.
- `.idxmax()` returns the index of the row with the maximum value. If there are multiple rows that achieve this value, then it will only return the index of the first occurrence.

The corresponding functions for the _minimum_ value exist as well:

- `.min()` calculates the minimum value.
- `.idxmin()` returns the index of the row with the minimum value. If there are multiple rows that achieve this value, then it will only return the index of the first occurrence.

In [11]:
df.fare.min()

0.0

Some passengers boarded the Titanic for free, apparently.

### Measures of Spread

The center of a quantitative variable only tells part of the story. For one, it tells us nothing about how spread out the values are. Therefore, it is important to also report a measure of **spread**.

Let's investigate a few measures of spread that are built into `pandas`. For completeness, the formulas for these statistics are provided, where $x_1, ..., x_n$ represent the values and $\bar x$ their mean. But don't worry to much about the formulas if you understand the intuition.

The first statistic that might come to mind is the **mean absolute deviation**, or MAD. To calculate the MAD, you first calculate the difference between each observation and the mean. Values below the mean will have a negative difference, while values above the mean will have a positive difference. We don't want the negative differences to cancel out the positive differences, since _all_ of them contribute to the spread. So we take the absolute value of all the differences and then average.

$$
\begin{align*}
\textrm{MAD} &= \textrm{mean of } |x_i - \bar x| \\
&= \frac{1}{n} \sum_{i=1}^n |x_i - \bar x|
\end{align*}
$$

We can implement the MAD ourselves using the `.mean()` and `.abs()` functions.

In [12]:
# STEP 1: Calculate the difference between each fare and the mean.
(df.fare - df.fare.mean())

0       178.042021
1       118.254521
2       118.254521
3       118.254521
           ...    
1305    -18.841279
1306    -26.070479
1307    -26.070479
1308    -25.420479
Name: fare, Length: 1309, dtype: float64

In [13]:
# STEP 2: Calculate the absolute value of each difference.
(df.fare - df.fare.mean()).abs()

0       178.042021
1       118.254521
2       118.254521
3       118.254521
           ...    
1305     18.841279
1306     26.070479
1307     26.070479
1308     25.420479
Name: fare, Length: 1309, dtype: float64

In [14]:
# STEP 3: Take the mean of these absolute differences.
(df.fare - df.fare.mean()).abs().mean()

29.798712688793827

Notice that in Step 1, we subtracted a single value (`df.fare.mean()`) from a `pandas` `Series` (`df.fare`). A `Series` is like an array, and in most programming languages, subtracting a number from an array is a type mismatch. But `pandas`  automatically **broadcasted** the subtraction over each number in the `Series`.

The `.abs()` function in Step 2 is another example of broadcasting. The absolute value function is applied to each element of the `Series`.

The MAD is actually built into `pandas`, so there really is no reason to implement it from scratch, as we did above. Let's check that we get the same answer when we call the built-in function.

In [15]:
df.fare.mad()

29.798712688793827

Since the MAD is a mean of the absolute differences and the mean represents the "typical" value, we can interpret the MAD as saying that the "typical" fare is about £30 away from the average.

Another way to ensure that the negative and positive differences don't cancel is to square all the differences before averaging. This leads to the definition of **variance**.

$$\textrm{Variance} = \textrm{mean of } (x_i - \bar x)^2$$

We can implement the variance ourselves using the .mean() and power (`**`) functions. Again, notice how the subtraction and the power function are broadcast over the elements of the `Series`.

In [16]:
((df.fare - df.fare.mean()) ** 2).mean()

2676.911603536706

Alternatively, we can simply call the `.var()` function in `pandas`.

In [17]:
df.fare.var()

2678.9597378928934

You might be surprised that `.var()` produces a slightly different number. This is because `pandas` divides by $n-1$ in calculating the mean of the squared differences, rather than $n$. That is, the formula that `pandas` uses is 

$$\text{Variance} = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar x)^2.$$

To force Pandas to divide by $n$, you can set `ddof=0`.

In [18]:
df.fare.var(ddof=0)

2676.911603536706

Now the value returned by `pandas` matches the value we obtained manually.

#### Why We Divide By $n-1$ (Optional)

Data is often a sample from some population. The point of calculating the variance of a sample is to be able to say something about the spread of the population.

To see why we divide by $n-1$ to measure the spread of a population, consider the extreme case where we have a sample of size $n=1$. What can we say about the spread of the population based on this one observation? Absolutely nothing! We need a sample of size at least $n=2$ to be able to say anything about the _spread_. Therefore, the variance is undefined when $n=1$. In order to make the variance not defined for $n=1$, we divide by $n-1$ so that we have $0/0$ when $n=1$. The variance formula above is only defined when $n \geq 2$.

The trouble with variance is that its units are wrong. If the original values $x_1, ..., x_n$ were in pounds, the variance would be in pounds _squared_. This is obvious if you simply look at the magnitude of the variance in the example above; the variance is in the _thousands_, even though the largest fare is just over £500.  To correct the units of variance, we take the square root to obtain a more interpretable measure of spread, called the **standard deviation**:

$$\textrm{SD} = \sqrt{\textrm{Variance}}.$$

In [19]:
df.fare.std()

51.758668239174135

We can interpret this standard deviation as saying: the "typical" fare is about £50 away from the average.

The standard deviation is the most widely used measure of spread, more common than the MAD. At first, this might seem odd. To calculate the standard deviation, we squared the differences from the mean, only to take a square root in the end. Why bother with this rigmarole, when we could just calculate absolute values instead?

The reasons for preferring the standard deviation are complicated. But the short answer is that the variance (which is the square of the standard deviation) is much nicer mathematically. If you know calculus, you might remember that the absolute value function does not have a derivative at 0. Therefore, the MAD is not _differentiable_, which makes it inconvenient mathematically. That doesn't necessarily mean that it's any worse as a measure of spread.

## Summary Statistics for Categorical Variables

Although there are many ways to summarize a quantitative variable, there is really only one way to summarize a categorical variable. Since a categorical variable can only take on a limited set of values, we can completely summarize the variable by reporting the frequencies of the different categories. The `pandas` function that produces this summary is `.value_counts()`.

In [20]:
embarked_counts = df.embarked.value_counts()
embarked_counts

S    914
C    270
Q    123
Name: embarked, dtype: int64

Note that the counts are sorted in decreasing order by default, so the first element corresponds to `top` in the summary produced by `.describe()`. Southampton was the most common point of embarkation. 

Since the counts are stored in a `pandas` `Series` indexed by category, we can extract a particular count using either label-based or position-based selection:

In [21]:
embarked_counts.loc["C"], embarked_counts.iloc[1]

(270, 270)

Instead of the _number_ of passengers embarking at each location, we might instead want to know the _percentage_ of passengers. To do this, divide the `Series` by the sum to turn the counts into **proportions**. (The term _proportion_ refers to a percentage when it is expressed as a number between 0 and 1, instead of between 0% and 100%.) Proportions must add up to 1, just as percentages must add up to 100%.

In [22]:
embarked_counts / embarked_counts.sum()

S    0.699311
C    0.206580
Q    0.094109
Name: embarked, dtype: float64

Notice the use of _broadcasting_ again; `embarked_counts` is a `Series`, but `embarked_counts.sum()` is a number. When a `Series` is divided by a number, the division is automatically applied to each element of the `Series`, producing another `Series`.

### Binary Categorical Variables

A binary categorical variable (i.e., a categorical variable with two categories) can be represented as a quantitative variable by coding one category as 1 and the other as 0.

In the Titanic data set, the `survived` variable has been coded this way. Each passenger either survived (1) or didn't (0).

In [23]:
df.survived

0       1
1       1
2       0
3       0
       ..
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 1309, dtype: int64

Although we can use `.value_counts()` to determine how many passengers survived:

In [24]:
df.survived.value_counts()

0    809
1    500
Name: survived, dtype: int64

we can also call `.sum()` and `.mean()` on this variable because the values are numeric.

What does `.sum()` do?

In [25]:
df.survived.sum()

500

`.sum()` returns the _number_ of ones. To see why, remember that this `Series` only 0s and 1s. Each 1 we encounter increments the sum by one, and each 0 contributes nothing to the sum. So when we add up all the numbers, we end up with the number of ones---or, in this example, the number of survivors.

What about `.mean()`?

In [26]:
df.survived.mean()

0.3819709702062643

`.mean()` returns the _proportion_ of ones. To see why, remember that the mean is the sum divided by the number of observations. The sum, as we have just discussed, is the number of 1s. Dividing this by the number of observations gives us the proportion of 1s---or, in this example, the proportion of survivors.

$$ \text{mean} = \frac{\text{sum}}{n} = \frac{\text{number of survivors}}{\text{number of passengers}} = \text{proportion of passengers who survived}.$$

## Summary Statistics for Other Variables?

In the last section, we noted that `name` is not a categorical variable because it does not take on a limited set of values. Hopefully, you now see why it was important to make this distinction. It does not make sense to analyze `name` like we analyzed `embarked` above. For example, if we calculate the frequency of each unique value in `name`, we don't learn much, since names generally do not repeat. 

In [27]:
df.name.value_counts()

Connolly, Miss. Kate                              2
Kelly, Mr. James                                  2
Cleaver, Miss. Alice                              1
Newsom, Miss. Helen Monypeny                      1
                                                 ..
Rothschild, Mrs. Martin (Elizabeth L. Barrett)    1
Cameron, Miss. Clear Annie                        1
Rugg, Miss. Emily                                 1
Roebling, Mr. Washington Augustus II              1
Name: name, Length: 1307, dtype: int64

That is why `name` was classified as an "other" variable. "Other" variables require additional processing before they can be summarized and analyzed. For example, if we extracted just the surnames from the `name` variable, then it might make sense to analyze this new variable as a categorical variable. The following case study shows how.

### Case Study: Extracting the Surname from the Names

We can extract the surnames from the names using the [built-in string processing functions](https://pandas.pydata.org/pandas-docs/stable/text.html), all of which are preceded by `.str`. The string processing function that will be most useful to us is `.str.split()`, which allows us to split each string in a `Series` by some sequence of characters.  (In other words, the `split()` function is _broadcast_ over the strings in the `Series`.) Since the surname and other names are separated by `", "`, we will split by `", "` to obtain two chunks, the first of which is the surname.

In [28]:
df.name.str.split(", ")

0              [Allen, Miss. Elisabeth Walton]
1             [Allison, Master. Hudson Trevor]
2               [Allison, Miss. Helen Loraine]
3       [Allison, Mr. Hudson Joshua Creighton]
                         ...                  
1305                   [Zabour, Miss. Thamine]
1306               [Zakarian, Mr. Mapriededer]
1307                     [Zakarian, Mr. Ortin]
1308                      [Zimmerman, Mr. Leo]
Name: name, Length: 1309, dtype: object

We can specify the option `expand=True` to get a `DataFrame` where each chunk is a separate column. The surnames are now in the first column.

In [29]:
df.name.str.split(", ", expand=True)

,0,1
0,Allen,Miss. Elisabeth Walton
1,Allison,Master. Hudson Trevor
2,Allison,Miss. Helen Loraine
3,Allison,Mr. Hudson Joshua Creighton
...,...,...
1305,Zabour,Miss. Thamine
1306,Zakarian,Mr. Mapriededer
1307,Zakarian,Mr. Ortin
1308,Zimmerman,Mr. Leo


Now we can select the surnames column (the column is named `0` in the `DataFrame`).

In [30]:
surnames = df.name.str.split(", ", expand=True)[0]
surnames

0           Allen
1         Allison
2         Allison
3         Allison
          ...    
1305       Zabour
1306     Zakarian
1307     Zakarian
1308    Zimmerman
Name: 0, Length: 1309, dtype: object

Since there are multiple passengers with the same surname, this is a categorical variable. We can use `.value_counts()` to find out which surnames were most common.

In [31]:
surnames.value_counts()

Sage         11
Andersson    11
Goodwin       8
Asplund       8
             ..
Mannion       1
Keefe         1
Bird          1
McGough       1
Name: 0, Length: 875, dtype: int64

Don't worry if the string processing is a bit over your head at this point. The purpose of this example was to illustrate how "other" variables can be wrangled into a form amenable to analysis.

## Summary

- Quantitative and categorical variables are summarized differently.
- For quantitative variables, we typically report a measure of center (e.g., mean, median, quantiles) and a measure of spread (e.g., variance, standard deviation, MAD).
- For categorical variables, we typically report the frequencies of the various categories, either as counts or as proportions.
- Other variables require additional processing before they can be analyzed.

# Exercises

All of the following exercises use the Tips data set (`tips.csv`).

**Exercise 1.** How many people were in the largest party served by the waiter? The smallest?

In [32]:
#Size of the party
df_tips = pd.read_csv("tips.csv")
print("df_tips: \n",df_tips)
print(df_tips["size"].value_counts())

df_tips: 
      obs  totbill   tip sex smoker  day   time  size
0      1    16.99  1.01   F     No  Sun  Night     2
1      2    10.34  1.66   M     No  Sun  Night     3
2      3    21.01  3.50   M     No  Sun  Night     3
3      4    23.68  3.31   M     No  Sun  Night     2
..   ...      ...   ...  ..    ...  ...    ...   ...
240  241    27.18  2.00   F    Yes  Sat  Night     2
241  242    22.67  2.00   M    Yes  Sat  Night     2
242  243    17.82  1.75   M     No  Sat  Night     2
243  244    18.78  3.00   F     No  Thu  Night     2

[244 rows x 8 columns]
2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


# Answer:
They are 24 people in the largest party served by the waiter.
<br>
They are 4 people in the smallest party served by the waiter.

**Exercise 2.** How could you use the `.quantile()` function to calculate the median? Check that your method works on an appropriate variable from the Tips data set.

In [33]:
# pth percentile: p percent of observations below it, (100 – p)%
# Percentiles divide the whole population into 100 groups where as quartiles divide the population into 4 groups
# p = 25: First Quartile or Lower quartile (LQ)
# p = 50: second quartile or Median
# p = 75: Third Quartile or Upper quartile (UQ)

# Showing some basic stats summary 
print("Total Bill Stat:\n", df_tips["totbill"].describe() )
print("Tips Stat:\n", df_tips["tip"].describe() )

# Finding the median in the tips and total bill
print("Total Bill Median:",df_tips["totbill"].median())
print("Tips Median:",df_tips["tip"].median())


print("Total Bill Quantile:", df_tips["totbill"].quantile(.75) )
print("Tips Quantile:", df_tips["tip"].quantile(.75) )

Total Bill Stat:
 count    244.000000
mean      19.785943
std        8.902412
min        3.070000
25%       13.347500
50%       17.795000
75%       24.127500
max       50.810000
Name: totbill, dtype: float64
Tips Stat:
 count    244.000000
mean       2.998279
std        1.383638
min        1.000000
25%        2.000000
50%        2.900000
75%        3.562500
max       10.000000
Name: tip, dtype: float64
Total Bill Median: 17.795
Tips Median: 2.9
Total Bill Quantile: 24.127499999999998
Tips Quantile: 3.5624999999999996


**Exercise 3.** Another measure of spread is the **interquartile range**, or IQR, defined as:

$$ \textrm{IQR} = \textrm{75th percentile} - \textrm{25th percentile}. $$

Measure the spread in the total bills by reporting the IQR.

In [34]:
df_tips_totbill_IQR = df_tips["totbill"].quantile(.75) - df_tips["totbill"].quantile(.25)
df_tips_totbill_IQR 

10.779999999999998

**Exercise 4.** Some people use MAD to refer to the **median absolute deviation**. The median absolute deviation is the same as the mean absolute deviation, but it uses the median instead of the mean:

$$\textrm{M(edian)AD} = \textrm{median of } |x_i - \textrm{median}|. $$

Calculate the median absolute deviation of the total bills. (The median absolute deviation is not built into Pandas, so you will have to implement it from scratch.)

In [35]:
# Same as the mean built from scratch code
# STEP 1: Calculate the difference between each total billd and the median
# x_i - median
(df_tips["totbill"] - df_tips["totbill"].median() )

# STEP 2: Calculate the absolute value of each difference.
#  | x_i - median |
(df_tips["totbill"] - df_tips["totbill"].median() ).abs()

# STEP 3: Take the median of these absolute differences.
# Median of | x_i - median |
(df_tips["totbill"] - df_tips["totbill"].median() ).abs().median()

5.03

**Exercise 5.** Who pays the bill more often: men or women?

In [36]:
#print("Total Bill\n", df_tips["totbill"])
#print("Sex:\n",df_tips["sex"])
df_tips_by_totbill_sex = df_tips[ ["totbill" , "sex"] ]
df_tips["sex"]

df_tips_by_sex_female = df_tips.set_index("sex").loc["F"]
df_tips_by_sex_male = df_tips.set_index("sex").loc["M"]

print("Females Total Bill Sum:",df_tips_by_sex_female["totbill"].sum() )
print("Males Total Bill Sum:",df_tips_by_sex_male["totbill"].sum() ) 
      
difference =  df_tips_by_sex_male["totbill"].sum() - df_tips_by_sex_female["totbill"].sum()
difference

Females Total Bill Sum: 1570.9499999999996
Males Total Bill Sum: 3256.8199999999997


1685.8700000000001

# Answer:
Men paid the bill more often than women; more than 50% that men paid more.